## Purpose

Export dataset from parquet to csv that I can use in Stata

I want: 1 row per person, with all the variables I need for my analysis

The variables are:

Outcome variables:
- `log_income`
- `log_wealth`
- `type_of_job_held`

Control variables
- `age`
- `gender`
- `HISCLASS`
- `Schooling`
- `MaritalStatus`

Inclusion variables
- `distance_from_parish_to_line`
- `touching_treated_parish`


## So what do I need to do to construct these variables?



Read in data

In [1]:
import pandas as pd
import pathlib
import numpy as np

root = pathlib.Path.cwd().parent

In [2]:
# read in census data
df_census_1930 = pd.read_parquet(root / "data/census/df_ref.parquet")
df_census_1930 = df_census_1930.rename(columns={'arkbild': 'name', 'scbkod': 'ref_code', 'fscbkod': 'ref_code_birth_parish'})

In [3]:
# string detect "yrke" in col names df_census_1930?
# detect "yrke" in column names of df_census_1930
yrke_cols = [col for col in df_census_1930.columns if "yrke" in col]

if yrke_cols:
    print("Columns containing 'yrke':")
    print(yrke_cols)
else:
    print("No columns containing 'yrke' found.")

Columns containing 'yrke':
['yrke.x', 'biyrke', 'yrke25', 'yrke.y']


In [4]:
# string detect "yrke" in col names df_census_1930?
# detect "yrke" in column names of df_census_1930
yrke_cols = [col for col in df_census_1930.columns if "income" in col]

if yrke_cols:
    print("Columns containing 'income':")
    print(yrke_cols)
else:
    print("No columns containing 'income' found.")

Columns containing 'yrke':
['income_imputed_median', 'income_imputed_mean', 'income_incl_zero']
